## Server actions

#### Note: Server actions are advised to to use only when you have a small app with not so many tables. For bigger projects Next.js API is adviced. Please check Next.js notes to learn more about it.

To create server action we have to create an asynchronous function. Let's say we will keep server action function in a separate file in lib folder.

In [ ]:
## actions.js

export const sayHello = async () => {
    "use server";  # we have to add "use server" directive, otherwise next.js will show an error ()
    ### If we have a multiple server actions functions we can add "use server" at the top, before imports!!!
    
    console.log("Hello World")
}

Now, when we know how the server action function looks like, we can create our form to get the data from input fields.

In [ ]:
## page.tsx (any page with the form);

import {addUser} from "@/lib/actions"  # import function (we will create it in the next step)

export const FormPage = () => {
    
    return (
        <div>
            <form onSubmit={addUser}>
                <input type="text" name="firstName" /> # name is obligatory!!!
                <input type="text" name="lastName" />
                <button onClick={addUser}>Add new user</button>
            </form>
        </div>
    )
}

Now, we can create our action function and "fetch" data from input fields.

In [ ]:
## actions.js

export const addUser = async (formData) => { # we add formData props which will be passed by a form
    
    ## we can get data from each field separately
    
    const firstName = formData.get("firstName")
    const lastName = formData.get("lastName")
    
    ## or we can destructurize formData
    const {firstName, lastName} = Object.fromEntries(formData);
    
}

###### Saving data

We can save the data as a new user in our database.

In [ ]:
## actions.js
import {connectToDb} from "@/lib/utils"
import { User } from "./models";
import { connectToDb } from "./utils";

export const addUser = async (formData) => { # we add formData props which will be passed by a form
    
    const {firstName, lastName} = Object.fromEntries(formData);
    
    try {
        connectToDb();
        const newUser = new User({
            firstName,
            lastName,
        })
        
        await newUser.save()  # saving new user
        revalidatePath("/users") # it will refresh the page where the data is fetched (even if caching is on);
    } catch (err) {
        return {error: "Something went wrong"}
    }
    
}

###### Deleting data

It's very similar to addUser.

In [ ]:
export const deleteUser = async (formData) => {

    const {userId} = Object.fromEntries(formData);

    try {
        connectToDb();
        await User.findByIdAndDelete(userId);
        console.log("deleted from db")
        revalidatePath("/users")
    } catch (error) {
        console.log(error)
        return {error: "Something went wrong"}
    }
}

When we want to delete all user's post when we delete him, we can do it like below.

In [ ]:
export const deleteUser = async (formData) => {

    const {id} = Object.fromEntries(formData);

    try {
        connectToDb();
        await Post.deleteMany({userId: id}) ## delete all posts with the userId property === id value
        await User.findByIdAndDelete(id)
        console.log("deleted from db");
        revalidatePath("/admin");
    } catch (error) {
        console.log(error)
        return {error: "Something went wrong"}
    }
}